In [1]:
import tensorflow as tf
import numpy as np
from voc_data_reader import VOCDataReader
from fcn import FCN
import matplotlib.pyplot as plt
import cv2
import os

%matplotlib inline

In [2]:
TRAINVAL_ROOT_DIR = '/root/PASCAL-VOC-Dataset/TrainVal'
TEST_ROOT_DIR = '/root/PASCAL-VOC-Dataset/TrainVal'
VGG_PARAMS_ROOT_DIR = '/root/tf-fcn/vgg-weights'

# fcn = FCN(TRAINVAL_ROOT_DIR, TEST_ROOT_DIR, VGG_PARAMS_ROOT_DIR)
dsr = VOCDataReader(TRAINVAL_ROOT_DIR, TEST_ROOT_DIR)

tb = dsr.next_train_batch(1)
images = tb['batch']
names = tb['names']
for i in images:
    img = i
img = np.reshape(img, [1, img.shape[0], img.shape[1], img.shape[2]])
img_path = os.path.join(TRAINVAL_ROOT_DIR, 'VOCdevkit/VOC2011/JPEGImages/{}.jpg'.format(names[0]))
img_unnormalized = cv2.imread(img_path)[:,:,::-1]

In [3]:
# Define a small network
inp = tf.placeholder(dtype=tf.float32)

conv_wts = tf.Variable(np.random.randn(3, 3, 3, 10), name = 'conv_wts', dtype=tf.float32)
conv1 = tf.nn.conv2d(inp, conv_wts, [1, 1, 1, 1], 'SAME')
pool1 = tf.nn.max_pool(conv1, [1, 2, 2, 1], [1, 2, 2, 1], 'SAME')
upconv_params = tf.Variable(np.random.randn(3, 3, 10, 10), name='upconv_wts', dtype=tf.float32)
upconv1 = tf.nn.conv2d_transpose(pool1, upconv_params, output_shape=[1, tf.shape(pool1)[1]*2, tf.shape(pool1)[2]*2, 10], strides=[1,2,2,1])


In [4]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    output = sess.run(upconv1, feed_dict = {inp: img})
    print(img.shape)
    print(output.shape)

(1, 375, 500, 3)
(1, 376, 500, 10)


In [ ]:
plt.imshow(img_unnormalized)

In [ ]:
plt.imshow(output[0][:,:,0])